In [5]:
import numpy as np
import horus as ho
# import pyoculus
# from mayavi import mlab
# from pyoculus.problems import CartesianBfield
from pyoculus.solvers import PoincarePlot
from pyoculus.solvers import FixedPoint
from simsopt.field import MagneticField

In [2]:
bs, bsh, (nfp, coils, ma, _) = ho.ncsx()

Setting up a problem

In [3]:
ps = ho.SimsoptBfieldProblem(ma.gamma()[0, 0], 0, 3, bs)

#### Poincare plot 
as in https://github.com/zhisong/pyoculus/blob/master/examples/two_waves.ipynb

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-9

# set up the Poincare plot
pparams = dict()
pparams["Rbegin"] = 1.0
pparams["Rend"] = 1.7
pparams["nPtrj"] = 10
pparams["nPpts"] = 100
pparams["zeta"] = 0.33 * np.pi

pplot = PoincarePlot(ps, pparams, integrator_params=iparams)
pdata = pplot.compute()
pplot.plot()

In [ ]:
pplot.compute_iota()
pplot.plot_iota()

#### Finding fixed point

In [16]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-7

pparams = dict()
pparams["nrestart"] = 0
pparams["Z"] = 0
pparams["tol"] = 1e-7
pparams['niter'] = 100

fp01 = FixedPoint(ps, pparams, integrator_params=iparams)
result01 = fp01.compute(guess=[1.527, 0.0], pp=3, qq=7, sbegin=1.5, send=1.9, tol = 1e-8)

[R,Z] : [1.5269993250240719, -0.00010262973645360548] - dtheta : 0.0006657053980969607
R : 1.5270057608464742
[R,Z] : [1.5270054078981177, -5.46862308122148e-05] - dtheta : -2.4681433785644913e-08
R : 1.527005760632846
[R,Z] : [1.5270054076726518, -5.468800803133927e-05] - dtheta : -1.4866330388940696e-11


In [17]:
# showing the resutlts
if result01 is not None:
    results = [list(p) for p in zip(result01.x, result01.y, result01.z)]
    print(results)

[[1.527005760632846, 0.0, 0.0], [1.3628334175644325, 0.0, -0.45231743094522103], [1.4409554131167368, 0.0, -0.4945674467790375], [1.6642136635137494, 0.0, -0.21187934399935102], [1.6642027866802638, 0.0, 0.21190993340436562], [1.4409372824072306, 0.0, 0.49457752367478125], [1.3628410006465215, 0.0, 0.4523017515552717], [1.5270054076726518, 0.0, -5.468800803133927e-05]]


In [ ]:
result01.GreenesResidue

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-7

pparams = dict()
pparams["nrestart"] = 0
# pparams["Z"] = 0
pparams["tol"] = 1e-7
pparams['niter'] = 10000

fp01 = FixedPoint(ps, pparams, integrator_params=iparams)
result01 = fp01.compute(guess=[1.527, -0.1], pp=3, qq=7, sbegin=1.5, send=1.9, tol = 1e-12, thetaf = False)

# showing the resutlts
if result01 is not None:
    results = [list(p) for p in zip(result01.x, result01.y, result01.z)]
    print(results)

We can check at the comet by hand

In [ ]:
ma.gamma()[0]

In [ ]:
phi = 0.33*np.pi
R = 1.15
startpoint = [R*np.cos(phi), R*np.sin(phi), 0]
startpoint = [1.9383899348191092, 0.0, 0.0]

In [ ]:
ma_gamma = ho.trace(bs, 8*2*np.pi, ma.gamma()[0])
# gamma = ho.trace(bs, 20*np.pi, startpoint)
#gamma2 = ho.trace(bs, 3*np.pi, startpoint)

In [ ]:
# mask = np.ones(gamma.shape[1], dtype=bool)
# for i, x in enumerate(gamma.T):
#     mask[i] = 0
#     if np.isin(gamma, gamma.T[mask]).all():
#         print(i)
#         break
#     mask[i] = 1

In [ ]:
#gammaX = ho.trace(bs, 21*np.pi, results[0])
gammaO = ho.trace(bs, 200*np.pi, [1.6282982137472521, 0.0, -0.5360026587795327])

In [ ]:
from mayavi import mlab
for coil in coils:
    coil.plot(engine="mayavi", show=False)

#mlab.plot3d(gammaX[0, :], gammaX[1, :], gammaX[2, :], tube_radius=0.01, color=(0, 0, 0))
mlab.plot3d(gammaO[0, :], gammaO[1, :], gammaO[2, :], tube_radius=0.01, color=(0, 0, 1))
#mlab.plot3d(ma_gamma[0, :], ma_gamma[1, :], ma_gamma[2, :], tube_radius=0.01, color=(1, 0, 0))

# Save the VTK object to a file
mlab.show()

#### Map of convergence

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-7

pparams = dict()
pparams["nrestart"] = 0
# pparams["Z"] = 0
pparams["tol"] = 1e-7
pparams['niter'] = 100

fp = FixedPoint(ps, pparams, integrator_params=iparams)

In [ ]:
R,Z = np.meshgrid(np.linspace(1.25, 1.8, 20), np.linspace(-0.6, 0.6, 20))

In [ ]:
assigned_to = list()
fixed_points = list()

In [ ]:
for i, (r, z) in enumerate(zip(R.flatten(), Z.flatten())):
    fp_result = fp.compute(guess=[r, z], pp=3, qq=7, sbegin=1.5, send=1.9, tol = 1e-8)

    if fp_result is not None:
        fp_result_xyz = np.array([fp_result.x[0], fp_result.y[0], fp_result.z[0]])
        assigned = False
        for j, fpt in enumerate(fixed_points):
            fpt_xyz = np.array([fpt.x[0], fpt.y[0], fpt.z[0]])
            if np.isclose(fp_result_xyz, fpt_xyz, atol=1e-3).all():
                assigned_to.append(j)
                assigned = True
        if not assigned:
            assigned_to.append(len(fixed_points))        
            fixed_points.append(fp_result)
    else:
        assigned_to.append(-1)

In [ ]:
assigned_to = np.array(assigned_to)+1

In [ ]:
from matplotlib.colors import TABLEAU_COLORS as colors
colors = list(colors.values())

cmap = np.array([list(int(colors[j].lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) for j in assigned_to])/255

In [ ]:
cmap = cmap.reshape(R.shape[0], R.shape[1], 3)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.pcolormesh(R, Z, cmap, shading='nearest')

for i, fpt in enumerate(fixed_points):
    if fpt.GreenesResidue < 0:
        marker = 'X'
    elif fpt.GreenesResidue > 0:
        marker = 'o'
    else:
        marker = 's'
    
    ax.scatter(fpt.x[0], fpt.z[0], c=colors[i+1], marker=marker, edgecolors='black', linewidths=1)

ax.set_aspect('equal')

#### Poincare plotter

In [ ]:
from scipy.integrate import solve_ivp

## for now 2d startpoint in the phi = 0 plane
RZstart = np.array([[1.59, 0],[1.60, 0]]) #, [1.54, 0]]) #, [1.6, 0], [1.7, 0], [1.8, 0]])

phis = [0, 0.33*np.pi]

In [ ]:
RZstart[:,0]

In [ ]:

def cyltocart(rphiz):
    r, phi, _ = rphiz
    return np.array([[np.cos(phi), -np.sin(phi), 0],
                     [-r*np.sin(phi), -r*np.cos(phi), 0],
                     [0, 0, 1]])

def carttocyl(xyz):
    x, y, _ = xyz
    r = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)

    return np.linalg.inv(cyltocart([r,phi,0]))

In [ ]:
import matplotlib.pyplot as plt

class PoincarePlane():
    def __init__(self, phi = 0, traces = None) -> None:
        if traces is None:
            traces = np.array([])
        else:
            if not isinstance(traces, np.ndarray):
                raise ValueError("traces must be a numpy array")
            elif not traces.shape[0] % 2 == 0:
                raise ValueError("traces must have 2*integer rows")
            
            self.traces = [traces[2*i:2*i+2,:] for i in range (traces.shape[0]//2)]
        
        self.phi = phi
    
    def plot(self):
        if len(self.traces) == 0:
            assert False, "No traces to plot"

        fig, ax = plt.subplots()
        for trace in self.traces:
            R, Z = trace
            plt.plot(R, Z, 'o', markersize=2)

        return fig, ax

def poincare(bs, RZstart, phis, **kwargs):
    options = {
        "phistart": 0, "nintersect": 10,
        "rtol": 1e-2, "atol": 1e-1,
    }
    options.update(kwargs)

    def Bfield_2D(t, xs):
        xs = xs.reshape(-1, 2)
        xs = np.hstack((xs[:,0].reshape(-1, 1), t*np.ones((xs.shape[0], 1)), xs[:,1].reshape(-1, 1)))
        bs.set_points_cyl(xs)

        Bs = bs.B()
        Bs_cyl = list()
        for xx, B in zip(xs,Bs):
            Bcyl = carttocyl(xx) @ B.reshape(3, -1)
            Bs_cyl.append([Bcyl[0,0]/Bcyl[1,0], Bcyl[2,0]/Bcyl[1,0]])

        return np.array(Bs_cyl).reshape(-1, 2).flatten()

    # setup the phis of the poincare sections
    phis = np.unique(np.mod(phis, 2*np.pi))
    phis.sort()

    # setup the evaluation points for those sections
    phi_evals = np.array([phis+2*np.pi*i for i in range(options['nintersect']+1)])
    print(phi_evals.flatten())

    print(RZstart.flatten())
    # solve the ODE
    out = solve_ivp(Bfield_2D, [options['phistart'], phi_evals[-1,-1]], RZstart.flatten(), t_eval=phi_evals.flatten(),
                rtol=options['rtol'], atol=options['atol'])
    
    # split the results into the different poincare sections using PoincarePlane objects
    ret_RZs = out.y
    
    print(out)
    
    nplanes = phis.size

    return [PoincarePlane(phis[i], ret_RZs[:,i::2]) for i in range(nplanes)]


In [ ]:
%timeit Bfield_2D(0, RZstart[0])

In [ ]:
def B(rphiz):
    bs.set_points_cyl(rphiz)
    return bs.B().reshape(3, -1)

%timeit B(np.random.rand(3, 1))

In [ ]:
res = poincare(bs, RZstart, phis)

In [ ]:
res[0].plot()